In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000025BCECB4280>, origin='C:\\Users\\MAFC4709\\Python_work\\RR_model_ECL_parallel\\src\\__init__.py', submodule_search_locations=['C:\\Users\\MAFC4709\\Python_work\\RR_model_ECL_parallel\\src'])


In [3]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)

✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}


## Load data build transition

In [21]:
# ===========================================================
# 📘 Roll Rate  - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================




# --- 2️⃣ Load dữ liệu thực tế ---
#sql_path = r"C:/Users/MAFC4709/Python_work/RR_model_ECL_parallel/sql/uns2.sql"

#df = load_data(sql_path, source="oracle")
df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
📦 Loading Parquet from: C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel\data\parquet
✅ Loaded 18,219,537 rows via pyarrow.dataset from C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel\data\parquet
✅ Dữ liệu có đủ các cột cần thiết: ['AGREEMENT_ID', 'MOB', 'STATE_MODEL', 'PRINCIPLE_OUTSTANDING', 'CUTOFF_DATE']


In [22]:
# import pandas as pd

# # Giả sử DataFrame của bạn tên là df
# df.to_parquet("CDL_2511.parquet", compression="snappy", index=False)


## Lấy data cutoff gần nhất

In [23]:
import sys
sys.path.append("src")

# Reload module khi sửa file
import importlib
import src.rollrate.ead_utils as ead_mod
importlib.reload(ead_mod)

# Import hàm EAD
from src.rollrate.ead_utils import (
    get_latest_ead,
    get_latest_snapshot,
    prepare_ead_input
)

# Test nhanh
latest_ead = get_latest_ead(df)
latest_ead.head()
df_current_raw = latest_ead[~latest_ead[CFG["state"]].isin(["PREPAY", "SOLDOUT"])]

## # 0. IMPORT & SETUP

In [24]:
# ============================================================

#    - Mục tiêu: Chuẩn bị module & config dùng trong toàn pipeline
#    - Đầu vào: các file .py đã tồn tại trong src/
#    - Đầu ra: alias tiện dùng: trans_mod, calib_mod, pd_fwd, CFG
#    - Đóng góp: Chuẩn hoá môi trường, tránh import rời rạc
# ============================================================

import pandas as pd
import numpy as np
import importlib

from src.config import CFG
from src.rollrate import transition as trans_mod
from src.rollrate import calibrate_markov_to_real_default as calib_mod
from src.rollrate import pd_forward as pd_fwd

# Reload để chắc chắn đang dùng version mới nhất trong src/
importlib.reload(trans_mod)
importlib.reload(calib_mod)
importlib.reload(pd_fwd)

loan  = CFG["loan"]
state = CFG["state"]
mob   = CFG["mob"]
ead   = CFG["ead"]
cutoff= CFG["cutoff"]

# 1. CHUẨN BỊ DỮ LIỆU LIFECYCLE CHO TRANSITION & CALIBRATION

In [25]:
# ============================================================

#    - Mục tiêu: Lấy panel lifecycle đủ sạch để:
#        + build transition Markov
#        + calibrate k-factor so với default thực tế
#    - Đầu vào: df (lifecycle đầy đủ nhiều cutoff)
#    - Đầu ra:
#        + df_trans: dùng cho compute_transition_by_mob (có thể full, lọc theo product)
#        + df_calib: lifecycle đã thu gọn (18M gần nhất) để chạy calibration tránh MemoryError
#    - Đóng góp:
#        + Giảm kích thước phù hợp RAM
#        + Tách rõ tập dùng cho transition vs tập dùng cho calibration
# ============================================================

# # Giả định: df đã có sẵn (lifecycle panel)
df[cutoff] = pd.to_datetime(df[cutoff])

# # 1.1. Lọc theo product nếu bạn chỉ làm cho CDLPIL
df_trans = df[df["PRODUCT_TYPE"] == "CDLPIL"].copy()

# # 1.2. Tạo df_calib = chỉ lấy 18 tháng gần nhất để calibration (giảm tải)
max_cutoff = df_trans[cutoff].max()
# df_calib = df_trans[df_trans[cutoff] >= (max_cutoff - pd.DateOffset(months=24))].copy()
df_calib = df
# # 1.3. Xoá cột DEFAULT_DATE cũ nếu có, tránh lỗi MergeError khi chạy lại nhiều lần
# drop_cols = [c for c in df_calib.columns if c.startswith("DEFAULT_DATE")]
# if drop_cols:
#     print("💥 Removing old DEFAULT_DATE columns from df_calib:", drop_cols)
#     df_calib = df_calib.drop(columns=drop_cols)

# print("👉 Số dòng df_trans (transition):", len(df_trans))
# print("👉 Số dòng df_calib (calibration):", len(df_calib))


# ============================================================

In [26]:
# 2. TÍNH TRANSITION MARKOV THEO MOB
#    - Mục tiêu: Xây ma trận P(mob, product, score) từ lifecycle
#    - Đầu vào: df_trans
#    - Đầu ra:
#        + matrices_by_mob: dict[product][mob][score]["P"]
#        + parent_fallback: ma trận fallback theo (product, score)
#    - Đóng góp:
#        + Đây là "engine" Markov gốc để forecast PD
# ============================================================

print("\n=== STEP 2: BUILD TRANSITION MATRICES ===")
matrices_by_mob, parent_fallback = trans_mod.compute_transition_by_mob(df_trans)


=== STEP 2: BUILD TRANSITION MATRICES ===
⚠️ Có 2 hàng có tổng weight = 0: ['PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 2 hàng có tổng weight = 0: ['PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 2 hàng có tổng weight = 0: ['PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 2 hàng có tổng weight = 0: ['PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 2 hàng có tổng weight = 0: ['PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 2 hàng có tổng weight = 0: ['PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 6 hàng có tổng weight = 0: ['DPD30+', 'DPD60+', 'DPD90+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 5 hàng có tổng weight = 0: ['DPD60+', 'DPD90+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 4 hàng có tổng weight = 0: ['DPD90+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 3 hàng có 

In [27]:
from src.rollrate.transition import load_transitions, save_transitions
save_transitions(matrices_by_mob, parent_fallback)  ## lưu  lại để không phải mất công chạy
#matrices_by_mob, parent_fallback = load_transitions() ## load lại để không phải mất công chạy

✔ Saved matrices_by_mob → backup_transition\matrices_by_mob.pkl
✔ Saved parent_fallback → backup_transition\parent_fallback.pkl


# 3. CALIBRATION MARKOV → REAL DEFAULT (k-factor)

In [28]:
calib_matrices_by_mob, calib_parent_fallback, calib_table = calib_mod.calibrate_markov_to_real_default(
    df_lifecycle=df_calib,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    horizon_months=12,
    min_ead_segment=1e4,
    k_floor=0.4,
    k_cap=3.0,
    use_mob_calib=True,                 # ⭐ bật calib theo MOB
    min_ead_segment_mob=3e3,            # ngưỡng nhỏ hơn cho từng MOB (tuỳ bạn)
)


In [29]:
calib_table

,LEVEL,SEG_PRODUCT,SEG_SCORE,MOB,TOTAL_EAD,REALIZED_PD_12M,PD_12M_MARKOV_AVG,K_FACTOR,NOTE
0,SEG,CDLPIL,500-,ALL,4.957134e+06,0.375193,0.382254,0.981529,k_raw=0.982
1,SEG,CDLPIL,550-,ALL,5.560468e+06,0.257591,0.255418,1.008506,k_raw=1.009
2,SEG,CDLPIL,600-,ALL,6.223103e+06,0.184442,0.174927,1.054393,k_raw=1.054
3,SEG,CDLPIL,650+,ALL,6.016514e+06,0.066031,0.057049,1.157454,k_raw=1.157
4,SEG,CDLPIL,650-,ALL,5.218978e+06,0.124179,0.112376,1.105030,k_raw=1.105
...,...,...,...,...,...,...,...,...,...
132,SEG_MOB,CDLPIL,OTHERS,17,2.230880e+02,0.126974,0.606934,0.991408,fallback_to_SEG
133,SEG_MOB,CDLPIL,OTHERS,18,1.753096e+02,0.159154,0.730033,0.991408,fallback_to_SEG
134,SEG_MOB,CDLPIL,OTHERS,19,1.175999e+02,0.235216,0.782383,0.991408,fallback_to_SEG
135,SEG_MOB,CDLPIL,OTHERS,20,5.361020e+01,0.385050,0.815238,0.991408,fallback_to_SEG


In [30]:
# ============================================================
# 4. CHUẨN BỊ SNAPSHOT HIỆN TẠI ĐỂ TÍNH PD & ECL
#    - Mục tiêu: Có snapshot dư nợ hiện tại (df_current_raw) sạch, không SOLDOUT
#    - Đầu vào:
#        + df_current_raw: đã lấy từ get_latest_ead / logic riêng của bạn
#        + (option) danh sách hợp đồng SOLDOUT nếu muốn loại thêm
#    - Đầu ra:
#        + df_current: bản copy để tính PD, LGD, ECL (không đụng vào raw)
#    - Đóng góp:
#        + Tách hẳn world "history" (lifecycle) và "now" (snapshot)
# ============================================================
df_soldout = pd.read_csv("C:/Users/MAFC4709/Python_work/RR_model_ECL_parallel/soldout_list.csv") 
print("\n=== STEP 4: PREPARE CURRENT SNAPSHOT ===")

# Giả định: df_current_raw đã có sẵn, gồm tối thiểu:
#   AGREEMENT_ID, CUTOFF_DATE, PRINCIPLE_OUTSTANDING, MOB, STATE_MODEL, PRODUCT_TYPE, (RISK_SCORE?)
df_current = df_current_raw.copy()

# Nếu bạn có 1 dataframe df_soldout_ids chỉ gồm AGREEMENT_ID cần loại:
df_current = df_current[~df_current[loan].isin(df_soldout[loan])].copy()
# Ở đây tạm comment:
# df_current = df_current[~df_current[loan].isin(df_soldout_ids[loan])].copy()

print("👉 Số dòng df_current (snapshot EAD):", len(df_current))





=== STEP 4: PREPARE CURRENT SNAPSHOT ===
👉 Số dòng df_current (snapshot EAD): 542573


# EAD 12M

In [31]:
sql_path = r"C:/Users/MAFC4709/Python_work/RR_model_ECL_parallel/sql/EAD.sql"

df_EAD12M = load_data(sql_path, source="oracle")

🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel\sql\EAD.sql
First 200 chars:
 select * from TV_MARKOV_EAD_12M
Params: {}


C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel\src\db.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


# LGD

In [32]:
import sys
sys.path.append("src")

# Reload module khi sửa file
import importlib
import src.rollrate.ead_utils as ead_mod
importlib.reload(ead_mod)

# Import hàm EAD
from src.rollrate.ead_utils import (
    get_latest_ead,
    get_latest_snapshot,
    prepare_ead_input
)

# Test nhanh
latest_ead = get_latest_ead(df)
latest_ead.head()
df_current_raw = latest_ead[~latest_ead[CFG["state"]].isin(["PREPAY", "SOLDOUT"])]

In [33]:
from src.rollrate.lgd_utils import run_lgd_pipeline

lgd_out = run_lgd_pipeline(
    data_path=r"C:/Users/MAFC4709/Python_work/RR_model_ECL/data/LGD2.parquet",
    use_loader=True
)

📦 Loading Parquet from: C:\Users\MAFC4709\Python_work\RR_model_ECL\data\LGD2.parquet
✅ Loaded 944,702 rows from LGD2.parquet
ℹ️ Added PRODUCT_TYPE='A' do không có trong dữ liệu.
📂 Loaded 944,702 rows.
✔ Preprocessed: 440,297 rows.
🎉 LGD Pipeline Completed!
Output folder: C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel\outputs\LGD


In [34]:
lgd_base = lgd_out["lgd_base"]

In [35]:
# ============================================================
# 6. MAP LGD (LGD_BASE → LGD_EFF) & TÍNH ECL_12M
#    - Mục tiêu:
#        + Gắn LGD theo PRODUCT_SEGMENT & MOB_BUCKET
#        + Tính LGD_EFF thực tế cho từng khoản vay
#        + Tính ECL_12M = PD_12M × LGD_EFF × EAD
#    - Đầu vào:
#        + df_current (có PD_12M, MOB, PRODUCT_TYPE)
#        + lgd_base: bảng LGD bạn đã có (PRODUCT_SEGMENT, MOB_BUCKET, LGD_BASE)
#    - Đầu ra:
#        + df_lgd: bảng đầy đủ để báo cáo (detail)
#    - Đóng góp:
#        + Ghép đủ 3 trụ cột PD – LGD – EAD → ECL_12M
# ============================================================

print("\n=== STEP 6: MAP LGD & COMPUTE ECL_12M ===")

# 6.1. Tính MOB_AT_DEFAULT ~ MOB + 6 (giả định sự kiện default trung bình tại 6/12M)
df_current["MOB_AT_DEFAULT"] = (df_current[mob] + 6).astype(int)
df_current["MOB_AT_DEFAULT"] = df_current["MOB_AT_DEFAULT"].clip(lower=0)

# 6.2. Chuẩn hoá LGD base
lgd = lgd_base.copy()
lgd["MOB_BUCKET_MIN"] = lgd["MOB_BUCKET"].str.split("-").str[0].astype(int)
lgd["MOB_BUCKET_MAX"] = lgd["MOB_BUCKET"].str.split("-").str[1].astype(int)

# 6.3. Merge theo PRODUCT_TYPE
df_lgd = df_current.merge(
    lgd,
    left_on="PRODUCT_TYPE",
    right_on="PRODUCT_SEGMENT",
    how="left"
)

# 6.4. Lọc đúng bucket MOB
mask_bucket = (
    (df_lgd["MOB_AT_DEFAULT"] >= df_lgd["MOB_BUCKET_MIN"]) &
    (df_lgd["MOB_AT_DEFAULT"] <= df_lgd["MOB_BUCKET_MAX"])
)
df_lgd = df_lgd[mask_bucket].copy()

# 6.5. LGD_EFF
df_lgd["LGD_EFF"] = df_lgd["LGD_BASE"].fillna(df_lgd["LGD_BASE"].mean())
df_lgd["LGD_EFF"].fillna(0.5, inplace=True)   # fallback cuối

# 6.6. Tính ECL_12M
df_lgd["ECL_12M"] = df_lgd["PD_12M"] * df_lgd["LGD_EFF"] * df_lgd[ead]
df_lgd["ECL_12M"].fillna(0.0, inplace=True)

print("Một vài dòng df_lgd (detail):")
display(df_lgd[[loan, state, mob, "PRODUCT_TYPE", "PD_12M", "LGD_EFF", ead, "ECL_12M"]].head())





=== STEP 6: MAP LGD & COMPUTE ECL_12M ===


C:\Users\MAFC4709\AppData\Local\Temp\ipykernel_21536\2694138416.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_lgd["LGD_EFF"].fillna(0.5, inplace=True)   # fallback cuối


KeyError: 'PD_12M'

In [37]:
from src.rollrate.ECL_final import compute_ecl_final

df_ecl = compute_ecl_final(
    df_current=df_lgd,
    df_sched=df_EAD12M,
    matrices_by_mob=calib_matrices_by_mob,
    parent_fallback=calib_parent_fallback,
    lgd_col="LGD_EFF"
)



=== COMPUTE ECL_FINAL (Optimized + Default Fix) ===
[LOG] schedule_map built for 551,799 loans in 40.673s
[LOG] PD cache built: 624 keys in 5.406s
[LOG] Computing ECL for each loan...
[LOG] Finished computing ECL for all loans in 13.348s


In [38]:
print("\n=== STEP 7: SUMMARY & EXPORT EXCEL (FINAL) ===")

df_report = df_ecl.copy()

ead = "PRINCIPLE_OUTSTANDING"
pd_col = "PD_12M_MARKOV"
state = "STATE_MODEL"
mob = "MOB"

# ============================================================
# 1. SUMMARY OVERALL
# ============================================================

total_ead = df_report[ead].sum()
total_ecl = df_report["ECL_12M"].sum()

weighted_pd  = (df_report[pd_col] * df_report[ead]).sum() / total_ead
weighted_lgd = (df_report["LGD_EFF"] * df_report[ead]).sum() / total_ead

summary = pd.DataFrame({
    "TOTAL_EAD": [total_ead],
    "TOTAL_ECL": [total_ecl],
    "WEIGHTED_PD": [weighted_pd],
    "WEIGHTED_LGD": [weighted_lgd],
    "ECL_RATE": [total_ecl / total_ead],
})


# ============================================================
# Helper: safe agg for pandas >=2.1
# ============================================================

def agg_block(df, group_cols):
    base = (
        df.groupby(group_cols)
        .agg({
            ead: "sum",
            "ECL_12M": "sum",
            pd_col: "mean",
        })
        .rename(columns={
            ead: "TOTAL_EAD",
            "ECL_12M": "TOTAL_ECL",
            pd_col: "AVG_PD",
        })
    )

    weighted_pd = (
        df.groupby(group_cols)
        .apply(lambda g: (g[pd_col] * g[ead]).sum() / g[ead].sum())
        .rename("W_PD")
    )

    weighted_lgd = (
        df.groupby(group_cols)
        .apply(lambda g: (g["LGD_EFF"] * g[ead]).sum() / g[ead].sum())
        .rename("W_LGD")
    )

    return base.join(weighted_pd).join(weighted_lgd)


# ============================================================
# 2. CLASSIC SUMMARIES
# ============================================================

report_by_product = agg_block(df_report, "PRODUCT_TYPE")
summary_by_mob    = agg_block(df_report, mob).reset_index()
summary_by_score  = agg_block(df_report, "RISK_SCORE").reset_index()
summary_by_state  = agg_block(df_report, state).reset_index()

df_report["SEGMENT"] = df_report["PRODUCT_TYPE"].astype(str) + "_" + df_report["RISK_SCORE"].astype(str)
summary_by_segment = agg_block(df_report, "SEGMENT")


# ============================================================
# 3. EXPORT DETAIL SHEET
# ============================================================

detail_cols = [
    "AGREEMENT_ID", "PRODUCT_TYPE", "RISK_SCORE",
    "STATE_MODEL", "MOB",
    "PRINCIPLE_OUTSTANDING",
    "LGD_EFF",
    "PD_12M_MARKOV", "PD_PROFILE_12M",
    "EAD_PROFILE_12M",
    "ECL_12M",
]

# Filter only available columns
detail_cols = [c for c in detail_cols if c in df_report.columns]


# ============================================================
# 4. EXPORT EXCEL
# ============================================================

output_file = "ECL_Report.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    df_report[detail_cols].to_excel(writer, sheet_name="Detail", index=False)
    summary.to_excel(writer, sheet_name="Summary", index=False)
    report_by_product.to_excel(writer, sheet_name="By_Product")
    summary_by_mob.to_excel(writer, sheet_name="By_MOB", index=False)
    summary_by_score.to_excel(writer, sheet_name="By_Score", index=False)
    summary_by_state.to_excel(writer, sheet_name="By_State", index=False)
    summary_by_segment.to_excel(writer, sheet_name="By_Segment")

print("✅ EXPORT ECL REPORT SUCCESS:", output_file)



=== STEP 7: SUMMARY & EXPORT EXCEL (FINAL) ===


C:\Users\MAFC4709\AppData\Local\Temp\ipykernel_21536\302317117.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g[pd_col] * g[ead]).sum() / g[ead].sum())
C:\Users\MAFC4709\AppData\Local\Temp\ipykernel_21536\302317117.py:56: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g["LGD_EFF"] * g[ead]).sum() / g[ead].sum())
C:\Users\MAFC4709\AppData\Local\Temp\ipykernel_21536\302317117.py

✅ EXPORT ECL REPORT SUCCESS: ECL_Report.xlsx


## Load XIRR

In [30]:
from src.config import PROJECT_ROOT

In [4]:
from src.rollrate.loader_ecl_xirr import load_ecl_xirr_folder

df_xirr = load_ecl_xirr_folder(incremental=False)




📂 Folder ECL_XIRR: C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel\data\ECL_XIRR
📁 Folder parquet: C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel\data\ECL_XIRR\parquet
📦 Đã có parquet cutoffs: [datetime.date(2025, 1, 31), datetime.date(2025, 2, 28), datetime.date(2025, 3, 31), datetime.date(2025, 4, 30), datetime.date(2025, 5, 31), datetime.date(2025, 6, 30), datetime.date(2025, 9, 30)]

📘 Xử lý file: ECL_28.02.2025 - PD From 01.03.2020-28.02.2025.xlsb
  📄 Sheet: ECL_28.02.25 | cutoff = 2025-02-28
  ➕ Rows load: 824,723
  💾 Saved → xirr_2025_02_28.parquet

📘 Xử lý file: ECL_30.04.2025 - PD From 01.05.2020-30.04.2025.xlsb
  📄 Sheet: ECL_30.04.25 | cutoff = 2025-04-30
  ➕ Rows load: 835,467
  💾 Saved → xirr_2025_04_30.parquet

📘 Xử lý file: ECL_30.06.2025 - PD From 01.07.2020-30.06.2025.xlsb
  📄 Sheet: ECL_30.06.25 | cutoff = 2025-06-30
  ➕ Rows load: 835,162
  💾 Saved → xirr_2025_06_30.parquet

📘 Xử lý file: ECL_30.09.2025 - PD From 01.10.2020-30.09.2025.xlsb
  📄 Sheet: ECL_30.

In [10]:
PROJECT_ROOT 

NameError: name 'PROJECT_ROOT' is not defined